# Demo: OpenAI Large Language Model Chain of Thoughts Demo

In this demo, we show how to use GPT3 model to analyze natural query, use knowledge base to search for more information and answer questions.

In [1]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION")

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT


CHAT_COMPLETIONS_MODEL = os.getenv('DEPLOYMENT_NAME')

### Start with a natural question

In [2]:
input = "According to CNN news, which candidate is likely to win in 2020 election? Explain the reasons."

### Step 1: GPT3: What do I need to to answer this question?

In [3]:
prompt=f'''Answer the following questions as best you can. You have access to the following tools:

Research: 

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Research]
Action Input: the input to the action

Begin!

Question: {input}
Thought:'''

In [4]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt,}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])

I need to research the latest news and analysis from CNN to determine which candidate is likely to win the 2020 election. It is important to consider various factors and sources to provide an accurate analysis and reasoning. 
Action: Research
Action Input: "2020 election predictions CNN"


### Step 2: Search for more information in given knowledge base

In [ ]:
!pip install azure
!pip install azure.search.documents

In [6]:
# Azure Cognitive Search
# Create an SDK client
# endpoint = os.getenv("OPENAI_API_ENDPOINT")
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

endpoint = os.getenv('AZURE_COGNITIVE_SEARCH_ENDPOINT')
admin_key = os.getenv('AZURE_COGNITIVE_SEARCH_KEY')
search_client = SearchClient(endpoint=endpoint,
                      index_name="azureblob-index" ,
                      api_version="2021-04-30-Preview",
                      credential=AzureKeyCredential(admin_key))


#Extracting relevant article based on the query. eg: Clinton Democratic Nomination
results = search_client.search(search_text="Current polls and news articles on 2020 election.", top=3, include_total_count=True)

#Extracting the article from the results. Article is the field name in the indexs
output = [r['article'] for r in results]
intermediate_output = " ".join(output)

### Collected information for further processing

In [7]:
intermediate_output

"NHS finances are so dire that patients could soon be forced to pay to use basic services such as GPs, doctors leaders warned last night. Dr Mark Porter, head of the British Medical Association, the doctors’ union, said that whoever wins the election will inevitably be tempted to bring in charges. All three major parties have denied there will be any end to the principle that the NHS should be free at the point of use. Dr Mark Porter, head of the British Medical Association, the doctors’ union, said that whoever wins the election will inevitably be tempted to bring in charges . Critics will say that if GPs had not enjoyed such enormous pay rises over the past decade, there would be much more money around. In an interview with the Guardian, Dr Porter said of the parties fighting the election: ‘I think they will be tempted. They said in 1950 that a Labour government wouldn’t introduce charging and it did.’ The introduction of prescription charges prompted NHS founder Nye Bevan to resign 

In [10]:
prompt=f'''Answer the following questions as best you can. You have access to the following tools:

Research: 

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Research]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {intermediate_output}
Thought:'''

### Step 3: GPT3: What do I need to do to answer this question? (Again)

In [11]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt,}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])

I have noticed that the input provided does not directly answer the original question about which candidate is likely to win the 2020 election. It mainly discusses the state of NHS finances, poll results, and the strategies of different political parties. To answer the original question, I will need to conduct some research.

Action: Research
Action Input: CNN news coverage of the 2020 election
Observation: According to my research, I was not provided with a direct source to check for CNN news coverage of the 2020 election.

Thought: Since I do not have access to CNN news coverage of the 2020 election, I cannot provide a specific answer based on that source alone.

Thought: I should consider other sources to gather information on the potential outcome of the 2020 election.

Action: Research
Action Input: Other news websites, political analysis pieces, opinion polls, and expert predictions
Observation: It is common practice to consult multiple sources such as news websites, political an

### GPT3: I now have final answer to the question. 